# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Ссылаясь на документацию, cтолбцы обозначают: 

* `сalls` — количество звонков;
* `minutes` — суммарная длительность звонков в минутах;
* `messages` — количество sms-сообщений;
* `mb_used` — израсходованный интернет-трафик в Мб;
* `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


*Заключение:* Итак при первой выгрузки данных, мы провели предварительный анализ данных, проверили данные на наличие пропусков (*из применяемых методов видно что пропусков нет*). Данные полностью устраивают и подходят для анализа. Теперь можно приступать собственно к построению моделей и ответам на вопросы проекта.

## Разбейте данные на выборки

Предварительно разделим таблицу на те данные на которых она будет обучатся модель и какое должна сделать предсказание.

In [5]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

Разделение на обучающую и тестовые выборки:

Данный разделены в соотношении:

 - обучающая 60%
 - валидационная 20%
- тестовая 20%

In [7]:
featur_trein, featur_val, target_trein, target_val = train_test_split(features, target, test_size=0.25, random_state=42)

In [8]:
featur_val, featur_test, target_val, target_test = train_test_split(featur_val, target_val, test_size=0.5, random_state=42)

In [9]:
len(featur_trein), len(featur_val), len(featur_test)

(2410, 402, 402)

Проверяем верно ли мы произвели "разделение" данных

**Вывод:** `train_test_split` помог создать одну большую и две не большие равные выборки. На основной группе будет происходить обучение, а на других тесты. Тестовые группы одинокого размера и с хаотичными данными. 

## Исследуйте модели

### модель Дерево Решений.

In [11]:
perfekt_accuracy = 0
depth = 0
best_model_tree = None

for i in range(1, 11):
    
    for x in (2, 11):
        model_tree = DecisionTreeClassifier(max_depth=i, max_leaf_nodes=x, random_state=42) #выбор алгоритма
        model_tree.fit(featur_trein, target_trein) #обучение
        predictions_tree = model_tree.predict(featur_val)#проверка на валидационной выборке
        score = accuracy_score(target_val, predictions_tree)

        if score > perfekt_accuracy: #отбор лучшего варианта
            perfekt_accuracy=score
            depth = i 
            best_model_tree = model_tree

perfekt_accuracy, depth, best_model_tree

(0.8208955223880597,
 4,
 DecisionTreeClassifier(max_depth=4, max_leaf_nodes=11, random_state=42))

для данной модели были выбраны 2 гиперпараметра. `max_depth` отвечает за размер дерева. `random_state` - помогает закрепить модель, для постоянства результата.
в данной модели происходит перебор деревьев и сохранение лучших параметров. 

### модель Логическая регрессия 

In [13]:
modell = LogisticRegression(random_state=42, solver='lbfgs', max_iter=500)
modell.fit(featur_trein, target_trein)
modell.predict(featur_val)
accuracy_logistic = modell.score(featur_val, target_val) 
accuracy_logistic

0.7039800995024875

Для данной модели был указан гиперпараметр `solver` позволяет выбрать алгоритм, который будет строить модель. Алгоритм `lbfgs` — один из самых распространённых для нашей модели он тоже подходит. `max_iter` - указывается количество итераций для обучения модели. 

Данная точность не подходит для нашей модели. 

### модель Случайный лес

In [14]:
best_model = None
best_result = 0
i = 0

for i in range(1, 11):

    model = RandomForestClassifier(random_state=42, n_estimators=i)
    model.fit(featur_trein, target_trein)
    result = model.score(featur_val, target_val)

    if result > best_result:
        best_model = model # сохраните наилучшую модель
        best_result = result
        i = i

best_result, best_model, i

(0.8407960199004975,
 RandomForestClassifier(n_estimators=10, random_state=42),
 10)

Гиперпараметр `n_estimators` указывается количество деревьев.

**Вывод:** у созданных моделей самый вывсокий результат `accuracy` показал Случайный лес 84% точность. Но нужно данную модель подвергнуть тестовой проверке, чтобы удостовериться насколько она хорошо способна делать предсказания.

Хуже всего справилась модель Логической регресси 70%. 

## Проверьте модель на тестовой выборке

### Проверка модели Дерево Решений на тестовых данных

In [15]:
print (f'Случайный Лес прошел проверку с долей правильных овттеов {perfekt_accuracy}, при размере дерева {depth}. \n' 
       f'Данный результат подходит нам, т.к. данная точность модели выше 0.75, проверяем \nполученную модель на тестовых данных.')

Случайный Лес прошел проверку с долей правильных овттеов 0.8208955223880597, при размере дерева 4. 
Данный результат подходит нам, т.к. данная точность модели выше 0.75, проверяем 
полученную модель на тестовых данных.


In [16]:
predictions_test = model_tree.predict(featur_test)#проверка на тестовой выборке
accuracy_score(target_test, predictions_test)

0.7935323383084577

### Проверка модели Логической регресси на тестовых данных

In [17]:
print (f'Логическая регрессия прошела проверку с долей правильных овттеов {accuracy_logistic} \n'
      'Данный результат не соответствеует результатам заданной модели')

Логическая регрессия прошела проверку с долей правильных овттеов 0.7039800995024875 
Данный результат не соответствеует результатам заданной модели


In [18]:
modell.predict(featur_test)
modell.score(featur_test, target_test)

0.7189054726368159

не целесообразно проверять данную модель, но ради эксперимента стоит оставить

### Проверка модели Случайный Лес на тестовых данных

In [19]:
print (f'Случайный Лес прошел проверку с долей правильных овттеов {best_result} c количеством деревьев {i}\n', 
      f'Данный результат подходит нам, т.к. данная точность модели выше 0.75, проверяем \nполученную модель на тестовых данных.')

Случайный Лес прошел проверку с долей правильных овттеов 0.8407960199004975 c количеством деревьев 10
 Данный результат подходит нам, т.к. данная точность модели выше 0.75, проверяем 
полученную модель на тестовых данных.


In [20]:
best_model.score(featur_test, target_test)

0.7910447761194029

**Вывод:** На тестовых данных лучший результат показал случайный лес. Данная проверка необходима, потому что созданная модель может показывать шикарные результаты на валидационной выборке, а на деле будет угадывать, данный нонсенс происходит в результате переобучения модели. 

После проверки качества нашей лучшей модели на тестовых данных и поняли что можем запустить её в промышленную эксплуатацию!

## (бонус) Проверьте модели на адекватность

Вариант - сравнить самый часто встречающийся класс в наших данных (это is_ultra == 0). Таких значений 2.229 в нашем датафрейме. Всего же значений в датафрейме 3.214. Значит самый часто встречающийся класс "занимает" 69% (2229 / 3214 == 0.693528313627878). Вот мы и получили контрольные данные для сравнительной оценки, построенной нами "лучшей" модели. Наша лучшая модель должна "побить" этот скор.

In [35]:
df.groupby('is_ultra').agg(['count'])['calls']

,count
is_ultra,
0,2229
1,985


In [37]:
2229/3214

0.693528313627878

p.s. (дополнительно, на перспективу!): можно было бы построить Confusion Matrix, чтобы детально посмотреть где ошибается модель.

# **Вывод:** 
В данной работе предварительно проанализированы данные таблицы. После чего стало понятно какие столбцы отнести к обучающей модели, а какие какой столбец нужно предсказать на основе полученных данных. Затем таблицу разделили на выборки. 
В работе проверились данные на разных моделях: Случайный лес, Дерево Решений, Логическая регрессия. 

Модель обучалась по следящим шагам: Определение необходимые гиперпараметры, обучение обученной модели скармливаем валидационную выборку и оцениваем результат, проверяем созданную модель на тестовых данных. Путем перебора и изменения параметров лучшая модель была выявлена – RandomForestClassifier (79%) точность оказалась самой высокой. 


## Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
